In [1]:
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('ggplot')

random_points = 100

In [ ]:
points = np.random.normal(size=[2, random_points]) + np.linspace(0, 20, random_points) # 0-20分100份 完美点
# size=2 x行y行都有误差
x = points[0, :]
y = points[1, :]
plt.scatter(x, y)